In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import git

import uproot as ut
import awkward as ak
import numpy as np
import math
import vector
import sympy as sp

import re
from tqdm import tqdm
import timeit
import re

sys.path.append( git.Repo('.', search_parent_directories=True).working_tree_dir )
from utils import *

In [2]:
%load_ext line_profiler
%load_ext memory_profiler

In [408]:
import torch
from torch_geometric.data import Data, DataLoader
from torch.nn.functional import softmax, relu
from torch_geometric.nn import dense_diff_pool
from torch_geometric.transforms import BaseTransform
from torch import Tensor
from typing import Tuple


In [115]:
from utils.torchUtils.cpp_geometric import layers

In [496]:
class HyperEdgeY(BaseTransform):
    def __init__(self, n_nodes=4):
        self.n_nodes=4
    def __call__(self, data : Data) -> Data:
        data.hyper_edge_index = torch.combinations(torch.arange(data.num_nodes),self.n_nodes).T
        cluster_y = (data.node_id+3)//4
        hyper_edge_y = cluster_y[data.hyper_edge_index.T]
        data.hyper_edge_y = 1*(hyper_edge_y == hyper_edge_y[:,0,None]).all(dim=-1)

        return data

In [501]:
import utils.torchUtils as gnn 
template = gnn.Dataset('eightb/pairing_methods/graph_net/data/template',make_template=True, transform=HyperEdgeY())
dataset = gnn.Dataset('eightb/pairing_methods/graph_net/data/MX_1200_MY_500-training',transform=template.transform)
# dataset = gnn.concat_dataset([f'eightb/pairing_methods/graph_net/data/{mass}-training' for mass in eightb.mass_list],transform=template.transform)

In [502]:
loader = DataLoader(dataset, batch_size=50, num_workers=8)

In [503]:
def get_n_classes(hyper_edge_y):
    true_hyper_edge = (hyper_edge_y == 1).sum()
    fake_hyper_edge = hyper_edge_y.shape[0] - true_hyper_edge
    return [fake_hyper_edge, true_hyper_edge]

In [479]:
class HyperEdgeLinear(torch.nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.linear = torch.nn.Linear(in_features,out_features)
    def __call__(self, x, edge_index, edge_attr, hyper_edge_index):
        hyper_edge_attr = x[hyper_edge_index.T]
        hyper_edge_attr = self.linear(hyper_edge_attr)
        hyper_edge_attr = hyper_edge_attr.max(dim=1)[0]
        return hyper_edge_attr

In [480]:
hyper_edge_linear = HyperEdgeLinear(5, 2)

In [481]:
hyper_edge_attr = hyper_edge_linear(batch.x, batch.edge_index, batch.edge_attr, batch.hyper_edge_index)

In [495]:
from torch.nn.functional import log_softmax, nll_loss

nll_loss(log_softmax(hyper_edge_attr,dim=-1), batch.hyper_edge_y, hyper_edge_weights)

tensor(0.7548, grad_fn=<NllLossBackward>)